
# Enriched Exoplanet Models: KOI + PDCSAP Light Curves

**Goal:** Build the *enriched* versions of your 4 baseline models by augmenting the KOI table with features derived from Kepler PDCSAP light curves (via `lightkurve`).  
This notebook does **only the enriched side**: it
1) Loads a KOI sample (top N rows from your Excel),
2) Fetches & preprocesses PDCSAP light curves per KIC,
3) Derives robust timeseries features (BLS periodogram, folded stats, basic LC stats),
4) Trains 4 models (GradientBoosting, XGBoost, LightGBM, RandomForest),
5) Exports a predictions CSV in the **Enriched Model** format you showed.

> Tip: Run in Google Colab for easy package setup and internet access (needed to fetch light curves).


In [1]:

# === If running in Colab, uncomment to install dependencies ===================
# !pip -q install lightkurve==2.4.0 astroquery==0.4.9 pywavelets==1.6.0 #                 scikit-learn==1.5.2 xgboost==2.1.1 lightgbm==4.5.0 #                 openpyxl==3.1.5

import os, json, re, warnings
warnings.filterwarnings("ignore")

from pathlib import Path
import numpy as np
import pandas as pd

# Timeseries / astronomy
import lightkurve as lk
from astropy.timeseries import BoxLeastSquares

# Optional wavelet scalogram
try:
    import pywt  # optional
except Exception:
    pywt = None

# ML
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

# Plotting
import matplotlib.pyplot as plt

# I/O
from datetime import datetime

print("Versions →",
      "lightkurve", lk.__version__,
      "| numpy", np.__version__,
      "| pandas", pd.__version__)


Versions → lightkurve 2.5.1 | numpy 2.2.0 | pandas 2.2.3


In [3]:
# ================== CONFIG ==================
# Path to your KOI Excel file
DATA_PATH = "data/Kepler Object of Interest.xlsx"   # <-- set to your file
TOP_N     = None                      # None = use ALL rows
OUT_DIR   = Path("./enriched_output")
OUT_DIR.mkdir(parents=True, exist_ok=True)

target_col = "koi_disposition"
feature_cols = [
    "koi_period", "koi_time0bk", "koi_duration", "koi_depth",
    "koi_prad", "koi_teq", "koi_steff", "koi_slogg", "koi_srad", "koi_kepmag"
]
required_id_cols = ["kepid", "kepoi_name", "kepler_name"]
required_cols = required_id_cols + feature_cols + [target_col]

# Train/test split
TEST_SIZE = 0.25
RANDOM_STATE = 42


def load_koi_sample(xlsx_path: str, top_n=None) -> pd.DataFrame:
    df = pd.read_excel(xlsx_path)
    # normalize columns
    df.columns = [str(c).strip().lower() for c in df.columns]

    missing = [c for c in required_cols if c not in df.columns]
    if missing:
        raise ValueError(f"Missing columns in Excel: {missing}")

    # harmonize label text
    df[target_col] = (df[target_col]
                      .astype(str)
                      .str.strip()
                      .str.replace('_', ' ')
                      .str.title())  # 'Confirmed','Candidate','False Positive'

    # select only needed columns once
    base = df[required_cols].copy()

    if isinstance(top_n, int) and top_n > 0:
        base = base.head(top_n)
    return base


koi_df = load_koi_sample(DATA_PATH, TOP_N)
print(f"✅ Loaded {len(koi_df)} KOI rows with required fields (TOP_N={TOP_N}).")
koi_df.head()



✅ Loaded 9564 KOI rows with required fields (TOP_N=None).


,kepid,kepoi_name,kepler_name,koi_period,koi_time0bk,koi_duration,koi_depth,koi_prad,koi_teq,koi_steff,koi_slogg,koi_srad,koi_kepmag,koi_disposition
0,10797460,K00752.01,Kepler-227 b,9.488036,170.538750,2.95750,615.8,2.26,793.0,5455.0,4.467,0.927,15.347,Confirmed
1,10797460,K00752.02,Kepler-227 c,54.418383,162.513840,4.50700,874.8,2.83,443.0,5455.0,4.467,0.927,15.347,Confirmed
2,10811496,K00753.01,NaN,19.899140,175.850252,1.78220,10829.0,14.60,638.0,5853.0,4.544,0.868,15.436,Candidate
3,10848459,K00754.01,NaN,1.736952,170.307565,2.40641,8079.2,33.46,1395.0,5805.0,4.564,0.791,15.597,False Positive
4,10854555,K00755.01,Kepler-664 b,2.525592,171.595550,1.65450,603.3,2.75,1406.0,6031.0,4.438,1.046,15.509,Confirmed


In [5]:
def load_koi_sample(xlsx_path: str, top_n: int | None = None) -> pd.DataFrame:
    df = pd.read_excel(xlsx_path)
    # normalize columns
    df.columns = [str(c).strip().lower() for c in df.columns]

    missing = [c for c in required_cols if c not in df.columns]
    if missing:
        raise ValueError(f"Missing columns in Excel: {missing}")

    # harmonize label text
    df[target_col] = (df[target_col]
                      .astype(str)
                      .str.strip()
                      .str.replace('_', ' ')
                      .str.title())  # 'Confirmed','Candidate','False Positive'

    # select only the required columns once (avoid join overlap)
    base = df[required_cols].copy()

    # if a top_n value is provided, limit the sample
    if isinstance(top_n, int) and top_n > 0:
        base = base.head(top_n)

    return base


koi_df = load_koi_sample(DATA_PATH, TOP_N)
print(f"✅ Loaded {len(koi_df)} KOI rows with required fields (TOP_N={TOP_N}).")
koi_df.head()


✅ Loaded 9564 KOI rows with required fields (TOP_N=None).


,kepid,kepoi_name,kepler_name,koi_period,koi_time0bk,koi_duration,koi_depth,koi_prad,koi_teq,koi_steff,koi_slogg,koi_srad,koi_kepmag,koi_disposition
0,10797460,K00752.01,Kepler-227 b,9.488036,170.538750,2.95750,615.8,2.26,793.0,5455.0,4.467,0.927,15.347,Confirmed
1,10797460,K00752.02,Kepler-227 c,54.418383,162.513840,4.50700,874.8,2.83,443.0,5455.0,4.467,0.927,15.347,Confirmed
2,10811496,K00753.01,NaN,19.899140,175.850252,1.78220,10829.0,14.60,638.0,5853.0,4.544,0.868,15.436,Candidate
3,10848459,K00754.01,NaN,1.736952,170.307565,2.40641,8079.2,33.46,1395.0,5805.0,4.564,0.791,15.597,False Positive
4,10854555,K00755.01,Kepler-664 b,2.525592,171.595550,1.65450,603.3,2.75,1406.0,6031.0,4.438,1.046,15.509,Confirmed


In [6]:

def fetch_pdcsap_lightcurve(kic_id: int):
    """Download & return a stitched, detrended PDCSAP light curve for a given Kepler KIC ID."""
    try:
        search = lk.search_lightcurvefile(f"KIC {kic_id}", mission="Kepler")
        if search is None or len(search) == 0:
            print(f"⚠️ No light curve files found for KIC {kic_id}.")
            return None

        lcf = search.download_all()
        if lcf is None:
            print(f"⚠️ Download failed for KIC {kic_id}.")
            return None

        # Clean and detrend the PDCSAP flux
        lc = lcf.PDCSAP_FLUX.stitch().remove_nans()
        if lc is None or len(lc.time) == 0:
            print(f"⚠️ Invalid or empty light curve for KIC {kic_id}.")
            return None

        lc = lc.flatten(window_length=401)  # detrend & normalize
        return lc

    except Exception as e:
        print(f"❌ Error fetching LC for KIC {kic_id}: {e}")
        return None


def compute_bls_periodogram(time, flux, min_period=0.5, max_period=100.0, n_samples=5000, duration_fraction=0.05):
    """Compute a Box Least Squares (BLS) periodogram and return periods, power, best period, best power."""
    try:
        periods = np.linspace(min_period, max_period, n_samples)
        bls = BoxLeastSquares(time, flux)
        res = bls.power(periods, duration_fraction)
        best_idx = int(np.nanargmax(res.power))
        return periods, res.power, float(periods[best_idx]), float(res.power[best_idx])
    except Exception as e:
        print(f"⚠️ BLS computation failed: {e}")
        return np.array([]), np.array([]), np.nan, np.nan


def fold_lightcurve(lc, period, t0):
    """Phase-fold the light curve by a given period and reference time."""
    try:
        folded = lc.fold(period=period, t0=t0)
        return np.array(folded.time.value), np.array(folded.flux.value)
    except Exception as e:
        print(f"⚠️ Folding failed: {e}")
        return np.array([]), np.array([])


def compute_scalogram(time, flux, wavelet="morl", scales=np.arange(1, 128)):
    """Compute a continuous wavelet transform (scalogram) from flux data."""
    if pywt is None:
        print("⚠️ PyWavelets not available — skipping scalogram.")
        return None
    try:
        coef, _ = pywt.cwt(flux, scales, wavelet)
        return coef
    except Exception as e:
        print(f"⚠️ Scalogram computation failed: {e}")
        return None


In [7]:
def derive_timeseries_features(lc, period=None, t0=None):
    """Extract statistical and signal features from a Kepler PDCSAP light curve."""
    # --- Return defaults if light curve is missing ---
    if lc is None or len(lc.time.value) == 0:
        return dict(
            lc_points=0,
            lc_flux_mean=np.nan,
            lc_flux_std=np.nan,
            lc_time_span_days=np.nan,
            bls_best_period=np.nan,
            bls_best_power=np.nan,
            folded_med_depth=np.nan,
            folded_odd_even_ratio=np.nan
        )

    # --- Extract base arrays ---
    try:
        t = np.array(lc.time.value)
        y = np.array(lc.flux.value)
    except Exception:
        return dict(
            lc_points=0,
            lc_flux_mean=np.nan,
            lc_flux_std=np.nan,
            lc_time_span_days=np.nan,
            bls_best_period=np.nan,
            bls_best_power=np.nan,
            folded_med_depth=np.nan,
            folded_odd_even_ratio=np.nan
        )

    # --- Basic statistics ---
    feats = dict(
        lc_points=int(len(y)),
        lc_flux_mean=float(np.nanmean(y)),
        lc_flux_std=float(np.nanstd(y)),
        lc_time_span_days=float(t.max() - t.min()) if len(t) > 1 else np.nan,
        bls_best_period=np.nan,
        bls_best_power=np.nan,
        folded_med_depth=np.nan,
        folded_odd_even_ratio=np.nan
    )

    # --- Compute BLS periodogram ---
    try:
        periods, power, best_p, best_pow = compute_bls_periodogram(t, y)
        if np.isfinite(best_p) and np.isfinite(best_pow):
            feats["bls_best_period"] = best_p
            feats["bls_best_power"] = best_pow
    except Exception as e:
        print(f"⚠️ BLS feature extraction failed: {e}")

    # --- Compute folded features if KOI period and t0 are provided ---
    if period is not None and t0 is not None and np.isfinite(period) and np.isfinite(t0):
        try:
            folded = lc.fold(period=period, t0=t0)
            ph = np.array(folded.time.value)
            yf = np.array(folded.flux.value)

            if len(ph) > 0 and len(yf) > 0:
                # median flux depth near transit (phase ~0)
                mask_in = np.abs(ph) < 0.1 * period
                feats["folded_med_depth"] = float(
                    np.nanmedian(yf[mask_in]) - np.nanmedian(yf[~mask_in])
                )

                # odd-even depth ratio to detect eclipsing binaries
                d1 = np.nanmedian(yf[(ph >= -0.25 * period) & (ph < 0.0 * period)])
                d2 = np.nanmedian(yf[(ph >= 0.0 * period) & (ph < 0.25 * period)])
                feats["folded_odd_even_ratio"] = (
                    float(d1 / d2) if np.isfinite(d1) and np.isfinite(d2) and d2 != 0 else np.nan
                )
        except Exception as e:
            print(f"⚠️ Folded feature computation failed: {e}")

    return feats



In [8]:
def enrich_koi_with_lc(koi_df: pd.DataFrame, save_artifacts: bool = True) -> pd.DataFrame:
    """
    For each KOI entry:
      - Fetch PDCSAP light curve (via Lightkurve)
      - Compute derived timeseries features (BLS, folded stats, LC stats)
      - Optionally save diagnostic plots
      - Return an enriched DataFrame combining original columns + features + artifact paths
    """
    records = []

    for idx, row in koi_df.iterrows():
        try:
            kic = int(row.get("kepid", np.nan))
            period = float(row.get("koi_period", np.nan))
            t0 = float(row.get("koi_time0bk", np.nan))
        except Exception:
            print(f"⚠️ Row {idx}: invalid numeric values, skipping.")
            continue

        print(f"\n🔭 Processing KIC {kic} | P ≈ {period:.3f} d | t0 ≈ {t0:.3f} BKJD")

        # --- Fetch PDCSAP light curve ---
        lc = fetch_pdcsap_lightcurve(kic)
        if lc is None:
            print(f"⚠️ Skipping KIC {kic} (no valid light curve).")
            feats = derive_timeseries_features(None)
            art = {"plots": {}, "status": "no_lightcurve"}
            rec = row.to_dict(); rec.update(feats); rec["artifacts"] = json.dumps(art)
            records.append(rec)
            continue

        # --- Derive light curve features ---
        feats = derive_timeseries_features(lc, period=period, t0=t0)
        art = {"plots": {}, "status": "ok"}

        # --- Save plots & diagnostics ---
        if save_artifacts:
            try:
                # Raw light curve
                fig = plt.figure()
                plt.plot(lc.time.value, lc.flux.value, linewidth=0.7)
                plt.xlabel("Time [BKJD]"); plt.ylabel("Normalized Flux")
                plt.title(f"KIC {kic} — PDCSAP (detrended)")
                pth = str((OUT_DIR / f"kic_{kic}_raw_lc.png").resolve())
                plt.tight_layout(); plt.savefig(pth, dpi=150); plt.close(fig)
                art["plots"]["raw_lc"] = pth

                # Phase-folded view
                ph, f = fold_lightcurve(lc, period, t0)
                if len(ph) > 0:
                    fig2 = plt.figure()
                    plt.scatter(ph, f, s=2, alpha=0.3)
                    plt.xlabel("Phase [days]"); plt.ylabel("Flux")
                    plt.title(f"KIC {kic} — Phase-folded ({period:.3f} d)")
                    pth2 = str((OUT_DIR / f"kic_{kic}_folded.png").resolve())
                    plt.tight_layout(); plt.savefig(pth2, dpi=150); plt.close(fig2)
                    art["plots"]["folded"] = pth2

                # BLS periodogram
                try:
                    periods, power, best_p, best_pow = compute_bls_periodogram(
                        lc.time.value, lc.flux.value
                    )
                    if len(periods) > 0:
                        feats["bls_best_period"] = float(best_p)
                        feats["bls_best_power"] = float(best_pow)
                        fig3 = plt.figure()
                        plt.plot(periods, power, linewidth=0.7)
                        plt.xlabel("Trial Period [days]"); plt.ylabel("BLS Power")
                        plt.title(f"KIC {kic} — BLS (best ≈ {best_p:.3f} d)")
                        pth3 = str((OUT_DIR / f"kic_{kic}_bls.png").resolve())
                        plt.tight_layout(); plt.savefig(pth3, dpi=150); plt.close(fig3)
                        art["plots"]["bls"] = pth3
                except Exception as e:
                    print(f"⚠️ BLS plot failed for KIC {kic}: {e}")

                # Optional: Wavelet scalogram
                try:
                    coef = compute_scalogram(lc.time.value, lc.flux.value)
                    if coef is not None:
                        fig4 = plt.figure()
                        plt.imshow(coef, aspect="auto", origin="lower")
                        plt.xlabel("Time Index"); plt.ylabel("Scale")
                        plt.title(f"KIC {kic} — Wavelet Scalogram")
                        pth4 = str((OUT_DIR / f"kic_{kic}_scalogram.png").resolve())
                        plt.tight_layout(); plt.savefig(pth4, dpi=150); plt.close(fig4)
                        art["plots"]["scalogram"] = pth4
                except Exception as e:
                    print(f"⚠️ Scalogram skipped for KIC {kic}: {e}")

            except Exception as e:
                print(f"⚠️ Artifact saving failed for KIC {kic}: {e}")

        # --- Merge results ---
        rec = row.to_dict()
        rec.update(feats)
        rec["artifacts"] = json.dumps(art)
        records.append(rec)

    # --- Combine enriched records ---
    enriched = pd.DataFrame(records)
    print(f"\n✅ Enrichment complete: {len(enriched)} KOIs processed.")
    return enriched


# --- Run enrichment ---
enriched_df = enrich_koi_with_lc(koi_df)
print("\nEnriched KOI (preview):")
display(enriched_df.head())



🔭 Processing KIC 10797460 | P ≈ 9.488 d | t0 ≈ 170.539 BKJD
⚠️ PyWavelets not available — skipping scalogram.

🔭 Processing KIC 10797460 | P ≈ 54.418 d | t0 ≈ 162.514 BKJD
⚠️ PyWavelets not available — skipping scalogram.

🔭 Processing KIC 10811496 | P ≈ 19.899 d | t0 ≈ 175.850 BKJD
⚠️ PyWavelets not available — skipping scalogram.

🔭 Processing KIC 10848459 | P ≈ 1.737 d | t0 ≈ 170.308 BKJD
⚠️ PyWavelets not available — skipping scalogram.

🔭 Processing KIC 10854555 | P ≈ 2.526 d | t0 ≈ 171.596 BKJD
⚠️ PyWavelets not available — skipping scalogram.

🔭 Processing KIC 10872983 | P ≈ 11.094 d | t0 ≈ 171.201 BKJD
⚠️ PyWavelets not available — skipping scalogram.

🔭 Processing KIC 10872983 | P ≈ 4.134 d | t0 ≈ 172.979 BKJD
⚠️ PyWavelets not available — skipping scalogram.

🔭 Processing KIC 10872983 | P ≈ 2.567 d | t0 ≈ 179.554 BKJD
⚠️ PyWavelets not available — skipping scalogram.

🔭 Processing KIC 6721123 | P ≈ 7.362 d | t0 ≈ 132.251 BKJD


KeyboardInterrupt: 

In [ ]:

# Select model features: original KOI numeric + derived LC features
derived_cols = ["lc_points","lc_flux_mean","lc_flux_std","lc_time_span_days",
                "bls_best_period","bls_best_power","folded_med_depth","folded_odd_even_ratio"]

model_features = feature_cols + derived_cols
X = enriched_df[model_features].copy()
y = enriched_df[target_col].copy()

# Numeric preprocessing (impute + scale)
numeric_processor = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

preproc = ColumnTransformer(
    transformers=[("num", numeric_processor, model_features)],
    remainder="drop"
)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE, stratify=y
)

print(f"Train size: {len(X_train)}  Test size: {len(X_test)}")


In [ ]:

models = {
    "Gradient Boosting EM": GradientBoostingClassifier(random_state=RANDOM_STATE),
    "XGBoost EM":          XGBClassifier(
        n_estimators=300, max_depth=5, learning_rate=0.05, subsample=0.9,
        colsample_bytree=0.9, objective="multi:softprob", eval_metric="mlogloss",
        random_state=RANDOM_STATE, n_jobs=-1
    ),
    "LightGBM EM":         LGBMClassifier(
        n_estimators=400, learning_rate=0.05, subsample=0.9,
        colsample_bytree=0.9, objective="multiclass", random_state=RANDOM_STATE
    ),
    "RandomForest EM":     RandomForestClassifier(
        n_estimators=600, max_depth=None, min_samples_split=2,
        random_state=RANDOM_STATE, n_jobs=-1
    ),
}


In [ ]:

# Fit models
fitted = {}
for name, clf in models.items():
    pipe = Pipeline(steps=[("pre", preproc), ("clf", clf)])
    pipe.fit(X_train, y_train)
    fitted[name] = pipe
    print(f"✅ Trained: {name}")

# Predictions table in your *Enriched Model* format
pred_df = pd.DataFrame({
    "Star System": enriched_df["kepid"].iloc[X_test.index].values,
    "Planet":      enriched_df["kepler_name"].iloc[X_test.index].values,
})

for name, pipe in fitted.items():
    pred = pipe.predict(X_test)
    pred_df[name] = pred

pred_df["Actual EM"] = y_test.values

# Consistent ordering of columns
col_order = ["Gradient Boosting EM","XGBoost EM","LightGBM EM","RandomForest EM","Actual EM"]
pred_df = pred_df[["Star System","Planet"] + col_order]

# Save CSV
csv_path = OUT_DIR / "koi_enriched_predictions.csv"
pred_df.to_csv(csv_path, index=False)
print("Saved predictions →", csv_path.resolve())

pred_df.head()


In [ ]:

# Quick text reports per model (on test set)
for name, pipe in fitted.items():
    y_pred = pipe.predict(X_test)
    print("\n=== Report:", name, "===")
    print(classification_report(y_test, y_pred, zero_division=0))
    print(confusion_matrix(y_test, y_pred))


In [ ]:

enriched_csv = OUT_DIR / f"koi_enriched_{len(enriched_df)}rows.csv"
enriched_df.to_csv(enriched_csv, index=False)
print("Saved enriched dataframe →", enriched_csv.resolve())
